In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch():
    model.train()
    param_hist = []
    for i, (x, y) in enumerate(tqdm(train_loader)):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
    return torch.tensor(np.mean(param_hist, 0))

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
model = nn.Sequential(
    nn.Conv2d(1, 32, 3),
    nn.ReLU(),
    nn.Conv2d(32, 32, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(4*4*64, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [7]:
initial_state = deepcopy(model.state_dict())

In [8]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")
loss_fn = nn.NLLLoss()

## Epoch

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1


/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Validation accuracy: 0.1644, validation loss: 2.3010
Epoch 2



Validation accuracy: 0.1486, validation loss: 2.2981
Epoch 3



Validation accuracy: 0.1982, validation loss: 2.2929
Epoch 4


...


Validation accuracy: 0.9703, validation loss: 0.0922
Epoch 27



Validation accuracy: 0.9722, validation loss: 0.0869
Epoch 28



Validation accuracy: 0.9740, validation loss: 0.0838
Epoch 29



Validation accuracy: 0.9743, validation loss: 0.0844
Epoch 30



Validation accuracy: 0.9735, validation loss: 0.0852


In [10]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9819, device='cuda:0'), tensor(0.0591, device='cuda:0'))



Valid: (tensor(0.9735, device='cuda:0'), tensor(0.0852, device='cuda:0'))


In [11]:
optimizer.accelerate()

In [12]:
optimizer.store_parameters()
model.cuda()

In [15]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9847, device='cuda:0'), tensor(0.0526, device='cuda:0'))



Valid: (tensor(0.9753, device='cuda:0'), tensor(0.0780, device='cuda:0'))


## Epoch average

In [16]:
model.load_state_dict(initial_state)
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch_avg")

In [17]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1698, validation loss: 2.3010
Epoch 2



Validation accuracy: 0.1468, validation loss: 2.2981
Epoch 3



Validation accuracy: 0.1940, validation loss: 2.2929
Epoch 4



Validation accuracy: 0.2494, validation loss: 2.2784
Epoch 5


...


Validation accuracy: 0.9717, validation loss: 0.0873
Epoch 27



Validation accuracy: 0.9745, validation loss: 0.0823
Epoch 28



Validation accuracy: 0.9735, validation loss: 0.0855
Epoch 29



Validation accuracy: 0.9743, validation loss: 0.0835
Epoch 30



Validation accuracy: 0.9762, validation loss: 0.0798


In [18]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))parameters_to_vector


Train: (tensor(0.9841, device='cuda:0'), tensor(0.0547, device='cuda:0'))



Valid: (tensor(0.9762, device='cuda:0'), tensor(0.0798, device='cuda:0'))


In [19]:
optimizer.accelerate()

In [20]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [21]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9768, device='cuda:0'), tensor(0.0914, device='cuda:0'))



Valid: (tensor(0.9695, device='cuda:0'), tensor(0.1075, device='cuda:0'))


## Split + epoch

In [23]:
model.load_state_dict(initial_state)
groups = [{"params": [param]} for param in model.parameters()]
optimizer = AcceleratedSGD(groups, 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")

In [24]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1641, validation loss: 2.3010
Epoch 2



Validation accuracy: 0.1517, validation loss: 2.2981
Epoch 3



Validation accuracy: 0.1999, validation loss: 2.2929
Epoch 4



Validation accuracy: 0.2554, validation loss: 2.2784
Epoch 5


...


Validation accuracy: 0.9743, validation loss: 0.0836
Epoch 27



Validation accuracy: 0.9739, validation loss: 0.0820
Epoch 28



Validation accuracy: 0.9704, validation loss: 0.0935
Epoch 29



Validation accuracy: 0.9712, validation loss: 0.0896
Epoch 30



Validation accuracy: 0.9728, validation loss: 0.0839


In [25]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9811, device='cuda:0'), tensor(0.0588, device='cuda:0'))



Valid: (tensor(0.9728, device='cuda:0'), tensor(0.0839, device='cuda:0'))


In [26]:
optimizer.accelerate()

In [27]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [28]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9830, device='cuda:0'), tensor(0.0553, device='cuda:0'))



Valid: (tensor(0.9753, device='cuda:0'), tensor(0.0844, device='cuda:0'))


## Split + epoch average

In [29]:
model.load_state_dict(initial_state)
groups = [{"params": [param]} for param in model.parameters()]
optimizer = AcceleratedSGD(groups, 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")

In [30]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1643, validation loss: 2.3010
Epoch 2



Validation accuracy: 0.1528, validation loss: 2.2981
Epoch 3



Validation accuracy: 0.1984, validation loss: 2.2929
Epoch 4



Validation accuracy: 0.2455, validation loss: 2.2785
Epoch 5


...


Validation accuracy: 0.9738, validation loss: 0.0834
Epoch 27



Validation accuracy: 0.9742, validation loss: 0.0830
Epoch 28



Validation accuracy: 0.9735, validation loss: 0.0831
Epoch 29



Validation accuracy: 0.9741, validation loss: 0.0845
Epoch 30



Validation accuracy: 0.9733, validation loss: 0.0881


In [31]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9794, device='cuda:0'), tensor(0.0647, device='cuda:0'))



Valid: (tensor(0.9733, device='cuda:0'), tensor(0.0881, device='cuda:0'))


In [32]:
optimizer.accelerate()

In [33]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [34]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9806, device='cuda:0'), tensor(0.0609, device='cuda:0'))



Valid: (tensor(0.9753, device='cuda:0'), tensor(0.0879, device='cuda:0'))
